In [ ]:
from sys import path
from os.path import dirname, abspath
path.append(dirname(dirname(abspath("__file__"))))

In [ ]:
from torchcrf import CRF
import torch
import torch.nn as nn
from torch_optimizer import Ranger
import torch.nn.functional as F
from transformers import get_cosine_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, random_split
from scripts.utils import *
from scripts.metrics import f1score
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from multiprocessing import cpu_count
from platform import system

pl.seed_everything(seed=42)

In [ ]:
LEARNING_RATE = 5e-1 # 3e-1 for bidirectional
BATCH_SIZE = 128
EPOCHS = 25
MAX_LEN = None
WEIGHT_DECAY = 1e-1
CELL_TYPE = "lstm"
MODEL_NAME = f"seq2seq-uni-{CELL_TYPE}-ner" # remove uni for bidirectional
N_JOBS = cpu_count() if system() != "Windows" else 0

TAG2IDX = {'B': 0, 'I': 1, 'O': 2, 'E': 3, 'S': 4, '<': 5, '>': 6, '$': 7}

In [ ]:
class SEQ2SEQ(pl.LightningModule):
    def __init__(self, 
                 input_dim, 
                 bidirectional=True, 
                 cell_type="lstm",
                 num_layers=3,
                 embed_dim=128,
                 dropout=0.5, 
                 cell_dim=128,
                 num_tags=len(TAG2IDX),
                 use_scheduler=True,
                 train_dataset=None,
                 val_dataset=None,
                 test_dataset=None):

        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=input_dim,
                                      embedding_dim=embed_dim,
                                      padding_idx=0)
        

        c = (2 if bidirectional else 1)
        if cell_type == "lstm":
            self.cell = nn.LSTM(input_size=embed_dim, 
                                hidden_size=cell_dim, 
                                dropout=dropout,
                                batch_first=True,
                                num_layers=num_layers, 
                                bidirectional=bidirectional)
        else:
            self.cell = nn.GRU(input_size=embed_dim, 
                                hidden_size=cell_dim, 
                                dropout=dropout,
                                batch_first=True,
                                num_layers=num_layers, 
                                bidirectional=bidirectional)

        self.fc = nn.Linear(cell_dim*c, num_tags)
        self.crf = CRF(num_tags=num_tags, batch_first=True)
        self.dropout = nn.Dropout(p=dropout)
        ## Hyperparameters ##
        self.use_scheduler = use_scheduler
        self.learning_rate = LEARNING_RATE
        self.batch_size = BATCH_SIZE
        self.weight_decay = WEIGHT_DECAY
        ## Datasets ##
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.test_dataset = test_dataset
        ## steps ##
        if self.use_scheduler: 
            self.total_steps = len(train_dataset) // self.batch_size

    def train_dataloader(self):
        return DataLoader(self.train_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          shuffle=True,
                          drop_last=False)


    def val_dataloader(self):
        return DataLoader(self.val_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          shuffle=False,
                          drop_last=False)


    def test_dataloader(self):
        return DataLoader(self.test_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          shuffle=False,
                          drop_last=False)
    

    def forward(self, input_ids):
        out, _ = self.cell(self.embedding(input_ids))
        out = F.relu(out)
        out = self.dropout(out)
        out = self.fc(out)
        return out


    def _shared_evaluation_step(self, batch, batch_idx):
        ids, masks, lbls = batch
        emissions = self(ids)
        loss = -self.crf(emissions, lbls, mask=masks)
        pred = self.crf.decode(emissions, mask=masks)
        r, p, f1 = f1score(lbls, pred)
        return loss, r, p, f1
     
        
    def training_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)
        return loss


    def validation_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)

    
    def test_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)


    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        ids, masks, _ = batch
        return self.crf.decode(self(ids), mask=masks)


    def configure_optimizers(self):           
        optimizer = Ranger(self.parameters(), 
                           lr=self.learning_rate,
                           weight_decay=self.weight_decay)

        if self.use_scheduler:
            scheduler = get_cosine_schedule_with_warmup(optimizer=optimizer,
                                                        num_warmup_steps=1,
                                                        num_training_steps=self.total_steps)
            lr_scheduler = {
                'scheduler': scheduler, 
                'interval': 'epoch', 
                'frequency': 1
            }
            return [optimizer], [lr_scheduler]
        else:
            return [optimizer]

In [ ]:
with open("../../data/full_vocab_ner.txt", mode="r", encoding="utf-8") as f:
    vocab = [s.strip() for s in f.readlines()]
    VOCAB2IDX = {v:k for (k, v) in enumerate(vocab)}

In [ ]:
encoded_input, masks, extended_labels = get_encoded_input("../../data/train_290818.txt", 
                                                          tag2idx=TAG2IDX,
                                                          vocab2idx=VOCAB2IDX,
                                                          maxlen=MAX_LEN)

L = len(extended_labels)

dataset = TensorDataset(torch.LongTensor(encoded_input),
                        torch.BoolTensor(masks),
                        torch.LongTensor(extended_labels))

train_sz, val_sz = L-int(0.1*L), int(0.1*L)
train_dataset, val_dataset = random_split(dataset, (train_sz, val_sz))                                                                                                                  

In [ ]:
encoded_input, masks, extended_labels = get_encoded_input("../../data/test_290818.txt", 
                                                          tag2idx=TAG2IDX,
                                                          vocab2idx=VOCAB2IDX,
                                                          maxlen=MAX_LEN)

test_dataset = TensorDataset(torch.LongTensor(encoded_input),
                             torch.BoolTensor(masks),
                             torch.LongTensor(extended_labels))

In [ ]:
model = SEQ2SEQ(input_dim=len(VOCAB2IDX),
                cell_type=CELL_TYPE,
                bidirectional=False,
                train_dataset=train_dataset,
                val_dataset=val_dataset,
                test_dataset=test_dataset,
                use_scheduler=True)

earlystopping_callback = EarlyStopping(monitor="val_f1score", 
                                       min_delta=1e-4, 
                                       patience=19, 
                                       mode="max")

checkpoint_callback = ModelCheckpoint(dirpath="../saved_weights",
                                      filename=MODEL_NAME,
                                      save_top_k=1, 
                                      mode="max",
                                      monitor="val_f1score",
                                      save_weights_only=True)

logger = TensorBoardLogger("../../tb_logs", name=MODEL_NAME)

trainer = pl.Trainer(accelerator="gpu",
                     max_epochs=EPOCHS,
                     precision=16,
                     amp_backend="apex",
                     logger=logger,
                     log_every_n_steps=1,
                     callbacks=[earlystopping_callback,
                                checkpoint_callback])

In [ ]:
trainer.fit(model)

In [ ]:
model.load_state_dict(torch.load(f"../saved_weights/{MODEL_NAME}.ckpt")["state_dict"])
trainer.test(model)